In [ ]:
%store -r model_data
%store -r s3_bucket
%store -r prefix

In [ ]:
from sagemaker import get_execution_role 

role = get_execution_role()

In [ ]:
from sagemaker.serverless import ServerlessInferenceConfig

serverless_config = ServerlessInferenceConfig(
  memory_size_in_mb=4096,
  max_concurrency=5,
)

In [ ]:
from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    model_data=model_data, 
    role=role, 
    source_dir="scripts",
    entry_point='inference.py', 
    framework_version='1.6.0',
    py_version="py3"
)

In [ ]:
%%time

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor = model.deploy(
    instance_type='ml.m5.xlarge', 
    initial_instance_count=1,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
    serverless_inference_config=serverless_config
)

In [ ]:
payload = {
    "text": "I love reading the book MLE on AWS!"
}

predictor.predict(payload)

In [ ]:
payload = {
    "text": "This is the worst spaghetti I've ever tasted"
}

predictor.predict(payload)

In [ ]:
predictor.delete_endpoint()